# Recurrent Neural Nets

This notebook explores RNN for flow prediction

In [11]:
import datetime
import calendar
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

# Load and prepare data

In [8]:
project_folder = '../../datasets/thorium-medium/'
flow = pd.read_csv(project_folder + 'flow1.csv', parse_dates=['time'])
flow = flow.set_index('time')['flow'].fillna(0)
flow = flow.resample('5T').pad()
rainfall = pd.read_csv(project_folder + 'rainfall1.csv', parse_dates=['time'])
rainfall = rainfall.set_index('time')['rainfall'].fillna(0)
rainfall = rainfall.resample('5T').pad()
flow_rain = pd.concat([flow, rainfall], axis=1).dropna()
print(flow_rain.head())
print(flow_rain.tail())

                           flow  rainfall
time                                     
2015-06-01 14:15:00  115.559998       0.0
2015-06-01 14:20:00  115.199997       0.0
2015-06-01 14:25:00  112.209999       0.0
2015-06-01 14:30:00  112.860001       0.0
2015-06-01 14:35:00  113.349998       0.0
                           flow  rainfall
time                                     
2017-11-10 14:20:00  107.830002       0.0
2017-11-10 14:25:00  107.459999       0.0
2017-11-10 14:30:00  106.919998       0.0
2017-11-10 14:35:00  105.559998       0.0
2017-11-10 14:40:00  104.940002       0.0


Replace DateTime with some features

In [9]:
flow_rain['day'] = flow_rain.index.map(lambda x : x.dayofyear)
flow_rain['minute'] = flow_rain.index.map(lambda x : x.hour*60 + x.minute)
input_data = flow_rain[['day', 'minute', 'flow', 'rainfall']]
input_data.head()

,day,minute,flow,rainfall
time,,,,
2015-06-01 14:15:00,152,855,115.559998,0.0
2015-06-01 14:20:00,152,860,115.199997,0.0
2015-06-01 14:25:00,152,865,112.209999,0.0
2015-06-01 14:30:00,152,870,112.860001,0.0
2015-06-01 14:35:00,152,875,113.349998,0.0


# Extract features

Keras RNN model as an input requires tensor with the following dimensions:
```
(batch_size, #steps, #features)
```
For this notebook we will use the following parameters

In [48]:
input_steps = 3*12   # 3 hours
output_steps = 24*12 # 24 hours
feature_count = 2    # time of the day and flow

## Create dataset

Convert input_data into input required by Keras and target value tensor

In [51]:
batch_size = input_data.shape[0] - input_steps
# We will use only 2 input features
a = input_data[['minute', 'flow']].values
X = np.array([a[i:i+input_steps, :] for i in range(batch_size)])
y = input_data.flow.values[input_steps:]
print(X.shape)
print(y.shape)

(257154, 36, 2)
(257154,)


# Create Keras model